In [1]:
from pathlib import Path
import os

yr = 2023
d = 2

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import re

id_extraction_re = re.compile(r'Game (?P<id>\d+):')
red_extraction_re = re.compile(r'.*?((?P<reds>\d+) red).*')
green_extraction_re = re.compile(r'.*?((?P<greens>\d+) green).*')
blue_extraction_re = re.compile(r'.*?((?P<blues>\d+) blue).*')


def format_input(inp):
  formatted_inp = {}
  for l in inp.split('\n'):
    id_results = id_extraction_re.match(l)
    cur_id = int(id_results['id'])
    cur_pulls = []
    for pull in l.split(';'):
      red_results = red_extraction_re.match(pull)
      green_results = green_extraction_re.match(pull)
      blue_results = blue_extraction_re.match(pull)
      cur_pulls.append((
                      int(red_results['reds'] if red_results is not None else 0),
                      int(green_results['greens'] if green_results is not None else 0),
                      int(blue_results['blues'] if blue_results is not None else 0)
                      ))
    formatted_inp[cur_id] = tuple(cur_pulls)

  return formatted_inp

In [3]:
def filter_possible_games(formatted_inp, red, green, blue):
  possible_game_ids = []
  for id, pulls in formatted_inp.items():
    red_possible = True
    green_possible = True
    blue_possible = True
    for pull in pulls:
      if pull[0] > red:
        red_possible = False
      if pull[1] > green:
        green_possible = False
      if pull[2] > blue:
        blue_possible = False
    if all([red_possible, green_possible, blue_possible]):
      possible_game_ids.append(id)
  return possible_game_ids


def calculate_powers(formatted_inp):
  powers = []
  for id, pulls in formatted_inp.items():
    min_red = None
    min_green = None
    min_blue = None
    for pull in pulls:
      if min_red is None or min_red < pull[0]:
        min_red = pull[0]
      if min_green is None or min_green < pull[1]:
        min_green = pull[1]
      if min_blue is None or min_blue < pull[2]:
        min_blue = pull[2]
    powers.append(min_red * min_green * min_blue)
  return powers

In [4]:
import time

t = time.time()

formatted_inp = format_input(inp)

print(sum(filter_possible_games(formatted_inp, red=12, green=13, blue=14)))
print(sum(calculate_powers(formatted_inp)))


print("\nRUNTIME: ", time.time()-t)

2331
71585

RUNTIME:  0.0019948482513427734
